In [ ]:
print("\n✓ Notebook execution completed successfully!")
print(f"All files saved with timestamp {timestamp} to avoid overwriting.")
print("\nIMPORTANT: Each execution creates a unique folder - no files will be overwritten!")

In [ ]:
# Generate comprehensive summary report
print("\n" + "="*70)
print("GRASS SPECIES 3-LEVEL GENERATION SUMMARY")
print(f"Execution Timestamp: {timestamp}")
print("="*70)

successful_prompts = 0
total_images = 0

# Summary by species (3 levels each)
for species in ['brachiaria', 'panicum', 'cynodon']:
    species_prompts = [k for k, v in all_generated_images.items() if v['species'] == species]
    species_successful = [k for k in species_prompts if all_generated_images[k]['images']]
    species_total_images = sum(len(all_generated_images[k]['images']) for k in species_prompts)
    
    print(f"\n{species.upper()} (3 Níveis Otimizados):")
    for prompt_key in species_prompts:
        data = all_generated_images[prompt_key]
        if data['images']:
            print(f"  ✓ {data['description']}: {len(data['images'])} images")
            successful_prompts += 1
            total_images += len(data['images'])
        else:
            print(f"  ✗ {data['description']}: Failed - {data.get('error', 'Unknown error')}")

print(f"\nRESULTADOS FINAIS:")
print(f"- Sistema: 3 níveis otimizados por espécie")
print(f"- Conjuntos de prompts bem-sucedidos: {successful_prompts}/{len(all_prompts_refined)}")
print(f"- Total de imagens geradas: {total_images}")
print(f"- Espécies processadas: {len(species_directories)}")
print(f"- Diretório de saída: {output_dir}")
print(f"- Timestamp único: {timestamp}")
print(f"- Pastas por espécie: {list(species_directories.keys())}")

print(f"\nVANTAGENS DO SISTEMA 3 NÍVEIS:")
print(f"- Maior consistência nas gerações")
print(f"- Melhor diferenciação entre níveis")
print(f"- Redução de sobreposição de arquivos")
print(f"- Nomenclatura única com timestamp")
print(f"- Foco em qualidade vs. quantidade")
print("="*70)

In [ ]:
# Save individual images with unique timestamp naming
def save_individual_images_with_timestamp(all_images, output_directory, timestamp):
    """
    Save each generated image individually with timestamp to avoid conflicts
    """
    saved_count = 0
    species_folders = {}
    
    # Create species directories with timestamp
    for prompt_key, data in all_images.items():
        species = data['species']
        if species not in species_folders:
            species_dir = os.path.join(output_directory, f"{species}_3levels_{timestamp}")
            os.makedirs(species_dir, exist_ok=True)
            species_folders[species] = species_dir
            print(f"Created directory: {species_dir}")
    
    # Save images with timestamp naming
    for prompt_key, data in all_images.items():
        if not data['images']:
            continue
        
        species = data['species']
        species_dir = species_folders[species]
        
        for i, image in enumerate(data['images']):
            filename = f"{prompt_key}_{timestamp}_img{i+1}.png"
            filepath = os.path.join(species_dir, filename)
            
            image.save(filepath)
            saved_count += 1
            print(f"Saved: {species}_3levels_{timestamp}/{filename}")
    
    print(f"\nTotal images saved: {saved_count}")
    print(f"Images organized in {len(species_folders)} species folders with timestamp {timestamp}")
    return saved_count, species_folders

# Save all images with timestamp
saved_images, species_directories = save_individual_images_with_timestamp(
    all_generated_images, output_dir, timestamp
)

In [ ]:
# Display all images in organized grid by species
def display_species_organized_grid(all_images):
    """
    Display all generated images organized by species in separate grids
    """
    species_list = ['brachiaria', 'panicum', 'cynodon']
    
    for species in species_list:
        # Get prompts for this species
        species_prompts = [(k, v) for k, v in all_images.items() 
                          if v['species'] == species and v['images']]
        
        if not species_prompts:
            print(f"No images to display for {species}")
            continue
        
        # Create subplot for this species
        rows = len(species_prompts)
        cols = 2  # 2 images per condition
        
        fig, axes = plt.subplots(rows, cols, figsize=(12, 4*rows))
        if rows == 1:
            axes = axes.reshape(1, -1)
        
        fig.suptitle(f"{species.capitalize()} - Sistema 3 Níveis (Timestamp: {timestamp})", fontsize=16, fontweight='bold')
        
        for i, (prompt_key, data) in enumerate(species_prompts):
            images = data['images']
            description = data['description']
            
            for j in range(min(cols, len(images))):
                axes[i, j].imshow(images[j])
                axes[i, j].set_title(f"{description} - Imagem {j+1}", fontsize=10)
                axes[i, j].axis('off')
            
            # Hide unused subplots
            for j in range(len(images), cols):
                axes[i, j].axis('off')
        
        plt.tight_layout()
        
        # Save species-specific grid
        grid_filename = f"{output_dir}/{species}_3levels_grid_{timestamp}.png"
        plt.savefig(grid_filename, dpi=300, bbox_inches='tight')
        plt.show()
        
        print(f"Saved {species} grid: {grid_filename}")

# Display organized grids
display_species_organized_grid(all_generated_images)

In [ ]:
# Generate all images for all species (3-level optimized system)
all_generated_images = {}
total_prompts = len(all_prompts_refined)
current_prompt = 0

print(f"Starting generation of {total_prompts} prompt sets (3 species × 3 levels each)")
print(f"Expected total images: {total_prompts * 2} (2 per prompt set)")
print("\n" + "="*60)

for prompt_key, prompt_data in all_prompts_refined.items():
    current_prompt += 1
    species = get_species_from_key(prompt_key)
    
    print(f"\nProcessing {current_prompt}/{total_prompts}: {prompt_key} ({species})")
    print("-" * 50)
    
    try:
        images = generate_species_images_optimized(prompt_data, num_images=2)
        all_generated_images[prompt_key] = {
            'images': images,
            'description': prompt_data['description'],
            'species': species
        }
        print(f"✓ Successfully generated 2 images for {prompt_key}")
        
    except Exception as e:
        print(f"✗ Error generating {prompt_key}: {e}")
        all_generated_images[prompt_key] = {
            'images': [],
            'description': prompt_data['description'],
            'species': species,
            'error': str(e)
        }

print(f"\n\n{'='*60}")
print("GENERATION COMPLETE!")
print(f"{'='*60}")

successful_generations = len([k for k, v in all_generated_images.items() if v['images']])
total_images_generated = sum(len(v['images']) for v in all_generated_images.values())

print(f"Generated images for {successful_generations}/{total_prompts} prompt sets.")
print(f"Total images: {total_images_generated}")

# Count by species
species_count = {}
for data in all_generated_images.values():
    species = data['species']
    if species not in species_count:
        species_count[species] = {'total': 0, 'successful': 0, 'images': 0}
    species_count[species]['total'] += 1
    if data['images']:
        species_count[species]['successful'] += 1
        species_count[species]['images'] += len(data['images'])

for species, counts in species_count.items():
    print(f"{species.capitalize()}: {counts['successful']}/{counts['total']} sets, {counts['images']} images")

In [ ]:
# Test generation with one prompt to verify the system
print("Testing optimized 3-level generation system...")

# Select a test prompt
test_prompt_key = 'brachiaria_optimal_health'
test_prompt_data = all_prompts_refined[test_prompt_key]

# Generate a test image
print("Generating test image with optimized parameters...")
test_images = generate_species_images_optimized(test_prompt_data, num_images=1)

# Display the test result
if test_images:
    plt.figure(figsize=(8, 8))
    plt.imshow(test_images[0])
    plt.title(f"Test Result: {test_prompt_data['description']}")
    plt.axis('off')
    plt.show()
    print("✓ Test generation successful - system ready!")
else:
    print("✗ Test generation failed")

In [ ]:
# Optimized generation function for 3-level system
def generate_species_images_optimized(prompt_data, num_images=2):
    """
    Optimized generation for 3-level system with consistent parameters
    """
    images = []
    
    positive_prompt = prompt_data['positive']
    negative_prompt = prompt_data['negative']
    
    print(f"Generating {num_images} images for: {prompt_data['description']}")
    print(f"Positive: {positive_prompt[:80]}...")
    print(f"Negative: {negative_prompt[:80]}...")
    
    for i in range(num_images):
        print(f"  Generating image {i+1}/{num_images}...")
        
        # Optimized parameters for consistency
        guidance_scale = 5.0  # Consistent guidance
        num_steps = 30        # Consistent steps
        seed = random.randint(0, 2**32 - 1)  # Random seed for variety
        
        print(f"    Parameters: guidance={guidance_scale}, steps={num_steps}, seed={seed}")
        
        with torch.inference_mode():
            generator = torch.Generator(device="cuda" if torch.cuda.is_available() else "cpu")
            generator.manual_seed(seed)
            
            image = pipe(
                prompt=positive_prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=num_steps,
                guidance_scale=guidance_scale,
                height=1024,
                width=1024,
                generator=generator
            ).images[0]
        
        images.append(image)
    
    return images

# Function to determine species from prompt key
def get_species_from_key(prompt_key):
    if prompt_key.startswith('brachiaria'):
        return 'brachiaria'
    elif prompt_key.startswith('panicum'):
        return 'panicum'
    elif prompt_key.startswith('cynodon'):
        return 'cynodon'
    else:
        return 'unknown'

print("✓ Optimized generation functions loaded!")

In [ ]:
# Create unique output directory with timestamp to avoid overwriting
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
date_folder = datetime.now().strftime("%Y-%m-%d")
output_dir = f"grass_3levels_generation_{timestamp}"
os.makedirs(output_dir, exist_ok=True)

print(f"Output directory: {output_dir}")
print(f"Date folder for organization: {date_folder}")
print(f"Timestamp: {timestamp}")
print("\n✓ Directory structure created - each execution will have unique folder!")

In [ ]:
# Sistema de 3 níveis otimizado para melhores resultados
brachiaria_prompts_refined = {
    'brachiaria_optimal_health': {
        'positive': (
            "perfect overhead view of prime brachiaria brizantha pasture, "
            "thick vibrant green tropical grass, dense uniform coverage, "
            "wide leaf blades in natural tufts, robust plant structure, "
            "excellent ground coverage, no bare soil visible, "
            "agricultural research photography, natural daylight, "
            "top-down perspective, healthy tropical grassland"
        ),
        'negative': (
            "sparse coverage, thin grass, bare patches, yellow grass, "
            "ground level view, side angle, artificial, synthetic, "
            "ornamental lawn, temperate species, flowers"
        ),
        'description': "Brachiaria - Saúde Ótima"
    },
    
    'brachiaria_good_condition': {
        'positive': (
            "aerial view of good condition brachiaria pasture, "
            "mostly green grass with natural color variation, "
            "good coverage with minor irregular patches, "
            "wide leaf tropical grass showing normal field variation, "
            "scientific documentation, natural lighting, "
            "overhead perspective, realistic pasture condition"
        ),
        'negative': (
            "severely damaged, mostly bare soil, dying grass, "
            "ground perspective, ornamental setting, artificial, "
            "temperate grass, decorative lawn"
        ),
        'description': "Brachiaria - Boa Condição"
    },
    
    'brachiaria_moderate_stress': {
        'positive': (
            "top-down view of brachiaria pasture under moderate stress, "
            "mixed green and lighter colored grass areas, "
            "uneven plant density, some small bare soil spots visible, "
            "tropical grass showing stress but still recognizable, "
            "natural variation in coverage, field research documentation, "
            "realistic pasture with environmental stress"
        ),
        'negative': (
            "completely healthy, uniform green, perfect coverage, "
            "ground level view, decorative grass, artificial, "
            "abstract patterns, unrecognizable vegetation"
        ),
        'description': "Brachiaria - Estresse Moderado"
    }
}

# Panicum com 3 níveis
panicum_prompts_refined = {
    'panicum_excellent_vigor': {
        'positive': (
            "overhead view of vigorous panicum maximum pasture, "
            "tall robust green tropical grass, wide leaf blades, "
            "dense natural clumping pattern, excellent coverage, "
            "healthy upright growth, thick stems visible, "
            "research quality photography, natural lighting"
        ),
        'negative': (
            "short grass, thin blades, sparse coverage, ground view, "
            "artificial, ornamental, temperate species"
        ),
        'description': "Panicum - Vigor Excelente"
    },
    
    'panicum_adequate_growth': {
        'positive': (
            "aerial view of adequate panicum grass field, "
            "mostly green tall grass with natural variation, "
            "good but not perfect coverage, mixed growth vigor, "
            "large tropical grass with normal field irregularities, "
            "scientific documentation, top-down perspective"
        ),
        'negative': (
            "severely stressed, mostly bare, dying vegetation, "
            "ground perspective, decorative setting, artificial"
        ),
        'description': "Panicum - Crescimento Adequado"
    },
    
    'panicum_declining_vigor': {
        'positive': (
            "overhead documentation of declining panicum pasture, "
            "tall grass with mixed green and pale areas, "
            "reduced plant density, some bare patches, "
            "tropical grass showing stress but maintaining structure, "
            "natural field conditions, research photography"
        ),
        'negative': (
            "perfect health, uniform coverage, decorative lawn, "
            "ground view, abstract patterns, unrecognizable"
        ),
        'description': "Panicum - Vigor em Declínio"
    }
}

# Cynodon com 3 níveis
cynodon_prompts_refined = {
    'cynodon_premium_condition': {
        'positive': (
            "perfect overhead view of premium cynodon dactylon, "
            "dense carpet-like grass coverage, uniform bright green, "
            "fine textured stoloniferous mat, excellent ground coverage, "
            "tight formation, no visible soil, research quality, "
            "natural lighting, top-down agricultural documentation"
        ),
        'negative': (
            "sparse coverage, bare patches, coarse texture, "
            "ground level view, artificial turf, ornamental"
        ),
        'description': "Cynodon - Condição Premium"
    },
    
    'cynodon_standard_quality': {
        'positive': (
            "aerial view of standard quality cynodon pasture, "
            "good carpet coverage with minor thin areas, "
            "mostly uniform green with natural color variation, "
            "stoloniferous grass with adequate density, "
            "scientific field documentation, overhead perspective"
        ),
        'negative': (
            "severely thin, mostly bare soil, dying grass, "
            "ground view, decorative setting, artificial"
        ),
        'description': "Cynodon - Qualidade Padrão"
    },
    
    'cynodon_thinning_coverage': {
        'positive': (
            "top-down view of cynodon with thinning coverage, "
            "patchy grass areas with visible thin spots, "
            "mixed density stoloniferous growth, some soil showing, "
            "declining but still recognizable cynodon pasture, "
            "natural field conditions, research documentation"
        ),
        'negative': (
            "thick perfect carpet, uniform coverage, decorative turf, "
            "ground perspective, abstract patterns, unrecognizable"
        ),
        'description': "Cynodon - Cobertura Rarefeita"
    }
}

# Combine all prompts
all_prompts_refined = {**brachiaria_prompts_refined, **panicum_prompts_refined, **cynodon_prompts_refined}

print(f"Loaded {len(brachiaria_prompts_refined)} Brachiaria prompts (3 levels)")
print(f"Loaded {len(panicum_prompts_refined)} Panicum prompts (3 levels)")
print(f"Loaded {len(cynodon_prompts_refined)} Cynodon prompts (3 levels)")
print(f"Total prompts: {len(all_prompts_refined)}")
print("✓ Sistema de 3 níveis otimizado carregado com sucesso!")

In [ ]:
# Load the Stable Diffusion 3.5 Medium model
print("Loading Stable Diffusion 3.5 Medium...")
pipe = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-medium",
    torch_dtype=torch.float16,
    device_map="balanced"
)

print("Model loaded successfully!")

In [ ]:
# Hugging Face login (required for SD 3.5 Medium)
login()

In [ ]:
# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Import libraries
import torch
from diffusers import StableDiffusion3Pipeline
from huggingface_hub import login
import matplotlib.pyplot as plt
from PIL import Image
import os
from datetime import datetime
import random
import shutil

# Stable Diffusion 3.5 Medium - Grass Species Generation (3 Níveis Otimizados)

Sistema otimizado para gerar imagens mais consistentes de espécies de gramíneas com apenas 3 níveis de degradação para cada espécie.

## Espécies e Condições Otimizadas:
- **Brachiaria brizantha**: 3 níveis (Saúde Ótima → Boa Condição → Estresse Moderado)
- **Panicum maximum**: 3 níveis (Vigor Excelente → Crescimento Adequado → Vigor em Declínio)
- **Cynodon dactylon**: 3 níveis (Condição Premium → Qualidade Padrão → Cobertura Rarefeita)

**Total**: 9 condições diferentes, 2 imagens cada = 18 imagens

**Melhorias nesta versão:**
- Sistema de 3 níveis para maior consistência
- Salvamento único por execução (evita sobreposição)
- Prompts refinados para melhor qualidade
- Nomenclatura timestamp para organização

In [ ]:
# Install required packages
!pip install diffusers transformers accelerate torch torchvision huggingface_hub --quiet